In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yyaaml1/dataset.yaml
/kaggle/input/smart-home-gesture-detection/Labels/opendoor.9df86d60-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/closedoor.ae788b48-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/opendoor.968831c8-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/closedoor.b85a5c72-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/closedoor.b5e332ac-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/closedoor.b71e5bb0-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/opendoor.9a45737a-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/closedoor.b230caa2-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/closedoor.b36bf496-eb9c-11ee-a03b-c869cd93666c.txt
/kaggle/input/smart-home-gesture-detection/Labels/opendoor.

**Dataset Splitting and Directory Creation**

In [5]:
from sklearn.model_selection import train_test_split
import shutil
import os

IMAGES_PATH = "/kaggle/input/smarthouse/collectedimages/images"
LABELS_PATH = "/kaggle/input/smarthouse/collectedimages/labels"

#Read Labels
labels = os.listdir(LABELS_PATH)

#Split data
train, test = train_test_split(labels, test_size=0.15, shuffle = True)
valid, test = train_test_split(test, test_size=0.2)
print(f"train: {len(train)}; valid {len(valid)}; test: {len(test)}")

os.makedirs("test/images")
os.makedirs("test/labels")
os.makedirs("train/images")
os.makedirs("train/labels")
os.makedirs("valid/images")
os.makedirs("valid/labels")


train: 101; valid 14; test: 4


**Delete Output Directory Code (if needed):**

In [ ]:
files2 = [file for file in os.listdir('/kaggle/working')]
print(files2)

In [4]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(folder_path)

OSError: [Errno 16] Device or resource busy: '/kaggle/working'

In [6]:
print(test)

['lightoff.b86a79a0-ee0c-11ee-add6-c869cd93666c.txt', 'lighton.4036b110-ee0c-11ee-add6-c869cd93666c.txt', 'closedoor.5010a35c-eed4-11ee-babd-c869cd93666c.txt', 'lightoff.be94916c-ee0c-11ee-add6-c869cd93666c.txt']


**Data Splitting and Organization Function**

In [7]:
def move_files_to_dir(files, dirname):
    for label_filename in files:
        image_filename = f"{label_filename[:-4]}.jpg"
        shutil.copy(f"{IMAGES_PATH}/{image_filename}", f"{dirname}/images/{image_filename}")
        shutil.copy(f"{LABELS_PATH}/{label_filename}", f"{dirname}/labels/{label_filename}")
        
#Move splits to folders
move_files_to_dir(train, "train")
move_files_to_dir(test, "test")
move_files_to_dir(valid, "valid")

print("Train Test Split Done!")

Train Test Split Done!


**Training YOLOv8n Model**

In [8]:
!pip install ultralytics

Here, we import the YOLO class from the ultralytics library, then initializes a YOLOv8n model using the pre-trained weights specified in the "yolov8n.pt". Subsequently, we train the model using the dataset provided in the "dataset.yaml" file, setting the image size to 640 pixels and training it for 200 epochs.

In [9]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(data='/kaggle/input/datayaml/dataset.yaml', imgsz=640, epochs=200)


100%|██████████| 6.23M/6.23M [00:00<00:00, 66.5MB/s]


Ultralytics YOLOv8.1.38 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/datayaml/dataset.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

100%|██████████| 755k/755k [00:00<00:00, 19.1MB/s]
2024-03-31 03:08:00,312	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-31 03:08:01,373	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-31 03:08:04.044346: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 03:08:04.044485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 03:08:04.203867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...


200 epochs completed in 0.117 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.38 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 13.71it/s]


                   all         14         14      0.989          1      0.995       0.84
              opendoor         14          4      0.986          1      0.995       0.85
             closedoor         14          1          1          1      0.995      0.796
               lighton         14          6      0.992          1      0.995      0.914
              lightoff         14          3      0.976          1      0.995      0.802
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train


lr/pg0,▂▄▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▂▄▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▂▄▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▆▅█████████████████████████████████████
metrics/mAP50-95(B),▁▆▄▇▇█▇▇▇▇██▇▇██▇▇▇▇█████▇██▇███████████
metrics/precision(B),▁▇██▇██▇█▇██████████████████████████████
metrics/recall(B),█▅▁▄█▇██████████████████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▄▄▄▄▄▄▄▄▃▃▃▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d163e40d1b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0